## 5-2　BERT を用いた文章の穴埋め

In [1]:
!pip install transformers==4.5.0 fugashi==1.1.0 ipadic==1.0.0

     |████████████████████████████████| 2.1 MB 523 kB/s            
     |████████████████████████████████| 486 kB 64.1 MB/s            
     |████████████████████████████████| 13.4 MB 54.8 MB/s            
  Preparing metadata (setup.py) ... - done
     |████████████████████████████████| 3.3 MB 52.8 MB/s            
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556723 sha256=9a7d9c6a449812041e3c6291a82b62bf656d230221ef6a990af836c294cbaea6
  Stored in directory: /root/.cache/pip/wheels/33/8b/99/cf0d27191876637cd3639a560f93aa982d7855ce826c94348b
Successfully built ipadic
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.11.6
    Uninstalling tokenizers-0.11.6:
      Successfully uninstalled tokenizers-0.11.6
  Attempting uninstall: transformers
    Found existing installation: transformers 4.16.2
    Uninstalling transformers-4.16.2:
      Successfully uninstalled transformers-4.16.2


In [2]:
import numpy as np
import torch
from transformers import BertJapaneseTokenizer, BertForMaskedLM

In [3]:
model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)
bert_mlm = BertForMaskedLM.from_pretrained(model_name)
bert_mlm = bert_mlm.cuda()

Downloading:   0%|          | 0.00/258k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/110 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/479 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
text = '今日は[MASK]へ行く。'
tokens = tokenizer.tokenize(text)
print(tokens)

['今日', 'は', '[MASK]', 'へ', '行く', '。']


In [5]:
input_ids = tokenizer.encode(text, return_tensors='pt')
input_ids = input_ids.cuda()
print(input_ids)

with torch.no_grad():
    output = bert_mlm(input_ids=input_ids)
    scores = output.logits

tensor([[   2, 3246,    9,    4,  118, 3488,    8,    3]], device='cuda:0')


In [6]:
mask_position = input_ids[0].tolist().index(4)
print(mask_position)

id_best = scores[0, mask_position].argmax(-1).item()
print(id_best)

token_best = tokenizer.convert_ids_to_tokens(id_best)
token_best = token_best.replace('##', '')

text = text.replace('[MASK]', token_best)

print(text)

3
391
今日は東京へ行く。


In [7]:
def predict_mask_topk(text, tokenizer, bert_mlm, num_topk):
    input_ids = tokenizer.encode(text, return_tensors='pt')
    
    input_ids = input_ids.cuda()
    with torch.no_grad():
        output = bert_mlm(input_ids=input_ids)
    scores = output.logits
    
    mask_position = input_ids[0].tolist().index(4)
    topk = scores[0, mask_position].topk(num_topk)
    ids_topk = topk.indices
    tokens_topk = tokenizer.convert_ids_to_tokens(ids_topk)
    scores_topk = topk.values.cpu().numpy()
    
    text_topk = []
    for token in tokens_topk:
        token = token.replace('##', '')
        text_topk.append(text.replace('[MASK]', token, 1))
        
    return text_topk, scores_topk

In [8]:
text = '今日は[MASK]へ行く。'
text_topk, _ = predict_mask_topk(text, tokenizer, bert_mlm, 10)
print(*text_topk, sep='\n')

今日は東京へ行く。
今日はハワイへ行く。
今日は学校へ行く。
今日はニューヨークへ行く。
今日はどこへ行く。
今日は空港へ行く。
今日はアメリカへ行く。
今日は病院へ行く。
今日はそこへ行く。
今日はロンドンへ行く。


`replace` の 3 つ目の引数には、置換する対象の数を入れる。

In [9]:
before = 'aaabbbccc'
after = before.replace('b', 'B', 1)
print(before)
print(after)

aaabbbccc
aaaBbbccc


In [10]:
def greedy_prediction(text, tokenizer, bert_mlm):
    for _ in range(text.count('[MASK]')):
        text = predict_mask_topk(text, tokenizer, bert_mlm, 1)[0][0]
    return text

text = '今日は[MASK][MASK]へ行く。'
greedy_prediction(text, tokenizer, bert_mlm)

'今日は、東京へ行く。'

In [11]:
text = '今日は[MASK][MASK][MASK][MASK][MASK]'
greedy_prediction(text, tokenizer, bert_mlm)

'今日は社会社会的な地位'

In [12]:
def beam_search(text, tokenizer, bert_mlm, num_topk):
    num_mask = text.count('[MASK]')
    text_topk = [text]
    scores_topk = np.array([0])
    for _ in range(num_mask):
        text_candidates = []
        score_candidates = []
        for text_mask, score in zip(text_topk, scores_topk):
            text_topk_inner, scores_topk_inner = predict_mask_topk(
                text_mask,
                tokenizer,
                bert_mlm,
                num_topk
            )
            text_candidates.extend(text_topk_inner)
            score_candidates.append(score + scores_topk_inner)
        
        score_candidates = np.hstack(score_candidates)
        idx_list = score_candidates.argsort()[::-1][:num_topk]
        text_topk = [text_candidates[idx] for idx in idx_list]
        scores_topk = score_candidates[idx_list]
        
    return text_topk

In [13]:
text = '今日は[MASK][MASK]へ行く。'
text_topk = beam_search(text, tokenizer, bert_mlm, 10)
print(*text_topk, sep='\n')

今日はお台場へ行く。
今日はお祭りへ行く。
今日はゲームセンターへ行く。
今日はお風呂へ行く。
今日はゲームショップへ行く。
今日は東京ディズニーランドへ行く。
今日はお店へ行く。
今日は同じ場所へ行く。
今日はあの場所へ行く。
今日は同じ学校へ行く。


`extend` は、末尾に別のリストやタプルを結合できる。

In [14]:
l = list(range(3))
print(l)

l.extend([100, 101, 102])
print(l)

[0, 1, 2]
[0, 1, 2, 100, 101, 102]


In [15]:
text = '今日は[MASK][MASK][MASK][MASK][MASK]'
text_topk = beam_search(text, tokenizer, bert_mlm, 10)
print(*text_topk, sep='\n')

今日は社会社会学会所属。
今日は社会社会学会会長。
今日は社会社会に属する。
今日は時代社会に属する。
今日は社会社会学会理事。
今日は時代社会にあたる。
今日は社会社会にある。
今日は社会社会学会会員。
今日は時代社会にある。
今日は社会社会になる。
